In [1]:
#!/usr/bin/env python
import os, json, argparse, datetime, math
import torch
import whisperx
import subprocess
import tempfile
import wave
import contextlib

In [2]:
def get_duration_sec(audio_path: str) -> float:
    try:
        with contextlib.closing(wave.open(audio_path,'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            return frames / float(rate)
    except Exception:
        # fallback con ffprobe si no es WAV
        try:
            out = subprocess.check_output([
                "ffprobe","-v","error","-show_entries","format=duration",
                "-of","default=noprint_wrappers=1:nokey=1", audio_path
            ]).decode().strip()
            return float(out)
        except Exception:
            return None

def merge_contiguous_turns(segments):
    """Une segmentos consecutivos del mismo speaker en un único turno."""
    if not segments: return []
    merged = []
    cur = dict(speaker=segments[0]["speaker"], start=segments[0]["start"], end=segments[0]["end"], text=segments[0]["text"])
    for seg in segments[1:]:
        if seg["speaker"] == cur["speaker"] and (seg["start"] - cur["end"]) <= 0.6:
            # pegar si están pegados o muy cercanos (0.6s)
            cur["end"] = seg["end"]
            cur["text"] += (" " if cur["text"] else "") + seg["text"]
        else:
            merged.append(cur)
            cur = dict(speaker=seg["speaker"], start=seg["start"], end=seg["end"], text=seg["text"])
    merged.append(cur)
    return merged

def guess_interviewer(speaker_stats, turns):
    # Heurística: entrevistador = quien hace más preguntas y habla menos tiempo.
    # 1) contar signos de interrogación y frases interrogativas por speaker
    q_words = ("¿", "?", "qué", "que", "quién", "quien", "cuándo", "cuando", "dónde", "donde",
               "por qué", "por que", "cómo", "como", "cuál", "cual", "cuáles", "cuales")
    q_score = {spk:0 for spk in speaker_stats}
    for t in turns:
        txt = t["text"].lower()
        if any(w in txt for w in q_words):
            q_score[t["speaker"]] += 1
    # 2) normalizar por tiempo total (quien pregunta más/tiempo) y habla menos
    best = None
    best_val = -1e9
    for spk, st in speaker_stats.items():
        time = st["total_sec"]
        asks = q_score.get(spk,0)
        # más preguntas por minuto y menos tiempo total => mayor score
        val = (asks / max(time,1e-6)) - 0.001*time
        if val > best_val:
            best_val = val
            best = spk
    return best

def build_qa(turns, interviewer):
    """Forma pares Q->A: pregunta del entrevistador y respuesta(s) hasta que el entrevistador hable de nuevo."""
    qa = []
    i = 0
    while i < len(turns):
        t = turns[i]
        if t["speaker"] == interviewer and ("?" in t["text"] or "¿" in t["text"]):
            q = {
                "q_speaker": interviewer,
                "q_start": t["start"],
                "q_end": t["end"],
                "question": t["text"].strip(),
                "answers": []
            }
            i += 1
            # recolecta todas las réplicas de otros speakers hasta que vuelva a hablar el entrevistador
            while i < len(turns) and turns[i]["speaker"] != interviewer:
                a = turns[i]
                if a["text"].strip():
                    q["answers"].append({
                        "a_speaker": a["speaker"],
                        "a_start": a["start"],
                        "a_end": a["end"],
                        "answer": a["text"].strip()
                    })
                i += 1
            qa.append(q)
        else:
            i += 1
    return qa

In [3]:
drive_path = os.getenv("DRIVE_PATH")
if not drive_path:
    raise ValueError("Debe definir la variable de entorno DRIVE_PATH con la ruta a Google Drive.")
# Filtrar solo archivos con extensiones de audio o video comunes
audio_video_exts = ('.mp3', '.aac', '.m4a', '.mp4', '.wav')
drive_files = [f for f in os.listdir(drive_path) if f.lower().endswith(audio_video_exts)]

In [4]:
import os
import torch
import datetime
import json
import torch

# Crear carpeta de salida
output_dir = os.path.join(drive_path, "output_json")
os.makedirs(output_dir, exist_ok=True)

# Parámetros configurables como variables
print("🔹 Configurando parámetros...")
model_name = "large-v3"
hf_token = os.getenv("HF_TOKEN")

# --- Elección de dispositivos por etapa ---
has_cuda = torch.cuda.is_available()
has_mps  = torch.backends.mps.is_available()

# ASR (WhisperX/Faster-Whisper via CTranslate2) -> NO soporta MPS
# asr_device      = "cuda" if has_cuda else "cpu"
# asr_compute     = "float16" if has_cuda else "int8"   # int8 va bien en CPU
asr_device      = "cpu"
asr_compute     = "int8"   # int8 va bien en CPU

# Alineación (MFA/CTranslate2) -> mismo que ASR
align_device    = asr_device

# Diarización (pyannote, PyTorch) -> puede usar MPS, si no CPU
diar_device     = "mps" if has_mps else ("cuda" if has_cuda else "cpu")

# (Opcional) hilos CPU para CTranslate2
asr_threads = max(1, os.cpu_count() // 2)

print(f"🔹 Usando device: {asr_device}")

language = "es"  # Cambia si necesitas otro idioma

🔹 Configurando parámetros...
🔹 Usando device: cpu


In [6]:
# Carga del Modelo
print("🔹 Cargando modelo WhisperX...")
model = whisperx.load_model(model_name, device=asr_device, compute_type=asr_compute, language=language)
#Carga del Modelo de Alineacion
print("🔹 Cargando modelo de alineación...")
align_model, metadata = whisperx.load_align_model(language_code=language, device=align_device)

🔹 Cargando modelo WhisperX...


C:\Users\Admn\AppData\Local\Programs\Python\Python310\lib\inspect.py:869: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\Admn\Desktop\Entrevistas\settlements-surveys\venv\lib\site-packages\whisperx\assets\pytorch_model.bin`


>>Performing voice activity detection using Pyannote...
Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu121. Bad things might happen unless you revert torch to 1.x.
🔹 Cargando modelo de alineación...


In [7]:
# --- Procesar todos los archivos ---
for fname in drive_files:
    print(f"\n🔹 Procesando archivo: {fname}")
    audio_path = os.path.join(drive_path, fname)
    out_path = os.path.join(output_dir, os.path.splitext(fname)[0] + ".json")

    try:
        # 1) Transcripción
        asr_result = model.transcribe(audio_path, batch_size=16)

        # 2) Alineación
        print("🔹 Realizando alineación...")
        aligned = whisperx.align(asr_result["segments"], align_model, metadata, audio_path, align_device)

        # 3) Diarización
        from whisperx.diarize import DiarizationPipeline
        diarize_pipeline = DiarizationPipeline(use_auth_token=hf_token, device=diar_device)
        diarize_segments = diarize_pipeline(audio_path)

        # 4) Asignar hablantes
        diarized = whisperx.assign_word_speakers(diarize_segments, aligned)

        # 5) Construcción de segmentos
        segs = []
        for seg in diarized["segments"]:
            if not seg.get("words"):
                continue
            words = [w for w in seg["words"] if "start" in w and "end" in w and "speaker" in w]
            if not words:
                continue
            current_spk = words[0]["speaker"]
            current_start = words[0]["start"]
            current_text = []
            for w in words:
                if w["speaker"] != current_spk:
                    segs.append({
                        "speaker": current_spk,
                        "start": current_start,
                        "end": prev_end,
                        "text": " ".join(current_text).strip()
                    })
                    current_spk = w["speaker"]
                    current_start = w["start"]
                    current_text = [w.get("word", "")]
                else:
                    current_text.append(w.get("word", ""))
                prev_end = w["end"]
            segs.append({
                "speaker": current_spk,
                "start": current_start,
                "end": prev_end,
                "text": " ".join(current_text).strip()
            })

        # 6) Unir tramos contiguos
        turns = merge_contiguous_turns(sorted(segs, key=lambda x: (x["start"], x["end"])))

        # 7) Estadísticas por hablante
        speaker_stats = {}
        for t in turns:
            d = t["end"] - t["start"]
            spk = t["speaker"]
            if spk not in speaker_stats:
                speaker_stats[spk] = {"total_sec": 0.0, "num_utts": 0}
            speaker_stats[spk]["total_sec"] += max(0.0, d)
            speaker_stats[spk]["num_utts"] += 1

        # 8) Detectar entrevistador
        interviewer = guess_interviewer(speaker_stats, turns)

        # 9) Preparar lista de hablantes
        speakers_list = []
        for spk, st in sorted(speaker_stats.items()):
            speakers_list.append({
                "id": spk,
                "total_sec": round(st["total_sec"], 3),
                "num_utts": st["num_utts"],
                "is_interviewer": (spk == interviewer)
            })

        # 10) Construir QA
        qa = build_qa(turns, interviewer)

        # 11) Metadata
        duration = get_duration_sec(audio_path) or 0
        result_json = {
            "meta": {
                "source_audio": os.path.abspath(audio_path),
                "language": asr_result['language'],
                "duration_sec": round(duration, 3),
                "created_utc": datetime.datetime.utcnow().replace(microsecond=0).isoformat() + "Z",
                "toolchain": {
                    "asr": f"whisperx-{model_name}",
                    "diarization": "pyannote",
                    "alignment": "mfa"
                }
            },
            "speakers": speakers_list,
            "turns": [
                {
                    "speaker": t["speaker"],
                    "start": round(t["start"], 3),
                    "end": round(t["end"], 3),
                    "text": t["text"]
                } for t in turns if t["text"]
            ],
            "qa": qa
        }

        print(f"🔹 Guardando resultados en {out_path} ...")
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(result_json, f, ensure_ascii=False, indent=2)

        print(f"✅ Listo: {out_path}")

    except Exception as e:
        print(f"❌ Error procesando {fname}: {str(e)}")


🔹 Procesando archivo: 60.m4a
🔹 Realizando alineación...


c:\Users\Admn\Desktop\Entrevistas\settlements-surveys\venv\lib\site-packages\pyannote\audio\utils\reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
c:\Users\Admn\Desktop\Entrevistas\settlements-surveys\venv\lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


🔹 Guardando resultados en C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\60.json ...
✅ Listo: C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\60.json

🔹 Procesando archivo: 69.m4a
🔹 Realizando alineación...


c:\Users\Admn\Desktop\Entrevistas\settlements-surveys\venv\lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


🔹 Guardando resultados en C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\69.json ...
✅ Listo: C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\69.json

🔹 Procesando archivo: 75 -1.m4a
🔹 Realizando alineación...


...

🔹 Guardando resultados en C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\Nicolas Montanares.json ...
✅ Listo: C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\Nicolas Montanares.json

🔹 Procesando archivo: Tamara Lara.mp4
🔹 Realizando alineación...


c:\Users\Admn\Desktop\Entrevistas\settlements-surveys\venv\lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


🔹 Guardando resultados en C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\Tamara Lara.json ...
✅ Listo: C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\Tamara Lara.json

🔹 Procesando archivo: Yam jamett.m4a
🔹 Realizando alineación...


c:\Users\Admn\Desktop\Entrevistas\settlements-surveys\venv\lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


🔹 Guardando resultados en C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\Yam jamett.json ...
✅ Listo: C:\Users\Admn\Desktop\Entrevistas\Audios\output_json\Yam jamett.json
